In [1]:
import pandas as pd 
import numpy as np 
import statsmodels as st

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm

# https://www.statsmodels.org/devel/examples/notebooks/generated/interactions_anova.html
# https://www.statsmodels.org/devel/example_formulas.html

In [2]:
# read in the metric files saved as csv
controls = pd.read_csv('../../DerivedData/extracted_diffusion_metrics_control_group.csv', index_col=0)
preterms = pd.read_csv('../../DerivedData/extracted_diffusion_metrics_preterm_group.csv', index_col=0)

#remove subthalamus
cols_to_keep = [col for col in controls.columns if 'Subthal' not in col]

### remove these for hemispheric comparisons! mention in the methods that these are not used in the mancova
cols_to_keep = [col for col in cols_to_keep if 'M1L-M1R' not in col]
cols_to_keep = [col for col in cols_to_keep if 'S1L-S1R' not in col]
cols_to_keep = [col for col in cols_to_keep if 'ParacentralL-ParacentralR' not in col]

### additionally remove the intra-hemispheric tracts involving Paracentral Region 
# => this is done below, keep S1L-M1L and S1R-M1R


controls = controls[cols_to_keep]
preterms = preterms[cols_to_keep]

### first get all regions pairs, metrics to be evaluated 
tract_names = np.unique(np.array([tract.split('_')[0] for tract in controls.columns[2:]]))
print('Number of evaluated bundles: {}'.format(len(tract_names)))
metrics = np.unique(np.array([tract.partition('_')[-1] for tract in controls.columns[2:]]))

### create pairing - as there is fewer preterms, use their IDs to find matches with controls 
matched = pd.read_csv('../../DerivedData/subject_matching.csv', index_col=0)
matched = matched[matched['preterm_ID'].isin(preterms['subject_id'].values)]

#sanity check: 
if len(preterms) == len(matched):
    print('Number of subject pairs: {}'.format(len(preterms)))
else:
    print('Someting happened with matching')
    
# get age at birth info for the subgrouping
ages = pd.read_csv('../../SourceData/release3_subject_info.tsv', sep='\t')

matched['preterm_birth_age'] = 0.
matched['control_birth_age'] = 0.
for i, row in matched.iterrows():
    matched.at[i, 'preterm_birth_age']  = ages[ages['participant_id '] == row['preterm_ID']+' ']['birth_age '].values[0]
    matched.at[i, 'control_birth_age']  = ages[ages['participant_id '] == row['matched_ID_with_outcome']+' ']['birth_age '].values[0]
    
### Create matched controls only 
controls = controls[controls.subject_id.isin(matched.matched_ID_with_outcome.values)]

concat_df = pd.concat([preterms, controls])

### create new column based on group belonging 
extreme_pairs = matched[matched.preterm_birth_age < 32][['preterm_ID','matched_ID_with_outcome']]
moderate_pairs = matched[matched.preterm_birth_age >= 32][['preterm_ID','matched_ID_with_outcome']]

concat_df['group'] = 0

concat_df.loc[concat_df['subject_id'].isin( extreme_pairs.preterm_ID.values), 'group'] = 0
concat_df.loc[concat_df['subject_id'].isin( moderate_pairs.preterm_ID.values), 'group'] = 1
concat_df.loc[concat_df['subject_id'].isin( extreme_pairs.matched_ID_with_outcome.values), 'group'] = 2
concat_df.loc[concat_df['subject_id'].isin( moderate_pairs.matched_ID_with_outcome.values), 'group'] = 3

### for metric = pull only the metric specific columns and transpose the df 
### rename the tract columns, encode into numbers 
### add Hemi 

metric = 'ODI_post'

metric_cols = [col+'_'+metric for col in tract_names]  ### and subject id
metric_cols.append('subject_id')
metric_cols.append('group')
concat_df = concat_df[metric_cols]

concat_df = pd.melt(concat_df, id_vars=['subject_id', 'group'], value_vars=[col for col in concat_df.columns if metric in col])

### add asymmetry 
concat_df['hemi'] = 0
concat_df.loc[concat_df['variable'].str.contains('R-'), 'hemi'] = 1


for i, row in concat_df.iterrows():
    
    b = row.variable.split('_')[0].split('-')    
    new = b[0][:-1] +'_'+ b[1][:-1]
    concat_df.loc[i,'tracts_combined'] = new
    
concat_df.drop(columns='variable', inplace=True)

### drop intra_hemi 
#to_drop = ['S1_M1', 'S1_Paracentral', 'M1_Paracentral']
to_drop = [ 'S1_Paracentral', 'M1_Paracentral']
concat_df = concat_df[~concat_df.tracts_combined.isin(to_drop)]

concat_df['tracts_combined'] = concat_df['tracts_combined'].astype('category')
concat_df['tract_cat'] = concat_df['tracts_combined'].cat.codes

df = concat_df[['subject_id', 'tract_cat', 'hemi', 'group', 'value']].dropna()

df['tract_cat'] = df['tract_cat'].astype(str)
df['hemi'] = df['hemi'].astype(str)
df['group'] = df['group'].astype(str)
# for statsmodels to treat variables as categorical

df.to_csv('../../DerivedData/for_jessica/{}_for_mancova_A1_no_intrahemi.csv'.format(metric))

Number of evaluated bundles: 30
Number of subject pairs: 59


In [314]:
#mod = smf.ols(formula='Lottery ~ Literacy + Wealth + Region', data=df)
mod = smf.ols(formula='value ~ tract_cat + hemi + group +  hemi*group + tract_cat*hemi + tract_cat*group + tract_cat*hemi*group', data=df, sub= df['subject_id'], wfactors=df[[ 'tract_cat', 'hemi']])

res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.796
Model:                            OLS   Adj. R-squared:                  0.788
Method:                 Least Squares   F-statistic:                     111.8
Date:                Thu, 27 Jan 2022   Prob (F-statistic):               0.00
Time:                        11:06:58   Log-Likelihood:                 8005.8
No. Observations:                3540   AIC:                        -1.577e+04
Df Residuals:                    3420   BIC:                        -1.503e+04
Df Model:                         119                                         
Covariance Type:            nonrobust                                         
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------


In [315]:
#Sum of squares
print(anova_lm(res, typ=1, robust='hc3'))

                          df    sum_sq   mean_sq           F        PR(>F)
tract_cat               14.0  8.494728  0.606766  922.241481  0.000000e+00
hemi                     1.0  0.005341  0.005341    8.117968  4.409067e-03
group                    3.0  0.157573  0.052524   79.833343  6.489806e-50
hemi:group               3.0  0.009186  0.003062    4.654245  2.993091e-03
tract_cat:hemi          14.0  0.018825  0.001345    2.043780  1.203874e-02
tract_cat:group         42.0  0.061687  0.001469    2.232378  9.928236e-06
tract_cat:hemi:group    42.0  0.006646  0.000158    0.240528  9.999999e-01
Residual              3420.0  2.250106  0.000658         NaN           NaN


# Data for the A2: Mancova with clinical variables and overall WM 

In [3]:
# read in the metric files saved as csv
controls = pd.read_csv('../../DerivedData/extracted_diffusion_metrics_control_group.csv', index_col=0)
preterms = pd.read_csv('../../DerivedData/extracted_diffusion_metrics_preterm_group.csv', index_col=0)

#remove subthalamus
cols_to_keep = [col for col in controls.columns if 'Subthal' not in col]
#cols_to_keep = [col for col in cols_to_keep if 'M1L-M1R' not in col]
#cols_to_keep = [col for col in cols_to_keep if 'S1L-S1R' not in col]
#cols_to_keep = [col for col in cols_to_keep if 'ParacentralL-ParacentralR' not in col]

controls = controls[cols_to_keep]
preterms = preterms[cols_to_keep]

### first get all regions pairs, metrics to be evaluated 
tract_names = np.unique(np.array([tract.split('_')[0] for tract in controls.columns[2:]]))
print('Number of evaluated bundles: {}'.format(len(tract_names)))
metrics = np.unique(np.array([tract.partition('_')[-1] for tract in controls.columns[2:]]))

### create pairing - as there is fewer preterms, use their IDs to find matches with controls 
matched = pd.read_csv('../../DerivedData/subject_matching.csv', index_col=0)
matched = matched[matched['preterm_ID'].isin(preterms['subject_id'].values)]

#sanity check: 
if len(preterms) == len(matched):
    print('Number of subject pairs: {}'.format(len(preterms)))
else:
    print('Someting happened with matching')
    
# get age at birth info for the subgrouping
ages = pd.read_csv('../../SourceData/release3_subject_info.tsv', sep='\t')

matched['preterm_birth_age'] = 0.
matched['control_birth_age'] = 0.
for i, row in matched.iterrows():
    matched.at[i, 'preterm_birth_age']  = ages[ages['participant_id '] == row['preterm_ID']+' ']['birth_age '].values[0]
    matched.at[i, 'control_birth_age']  = ages[ages['participant_id '] == row['matched_ID_with_outcome']+' ']['birth_age '].values[0]
    
### Create matched controls only 
controls = controls[controls.subject_id.isin(matched.matched_ID_with_outcome.values)]

concat_df = pd.concat([preterms, controls])

### create new column based on group belonging 
extreme_pairs = matched[matched.preterm_birth_age < 32][['preterm_ID','matched_ID_with_outcome']]
moderate_pairs = matched[matched.preterm_birth_age >= 32][['preterm_ID','matched_ID_with_outcome']]

concat_df['group'] = 0

concat_df.loc[concat_df['subject_id'].isin( extreme_pairs.preterm_ID.values), 'group'] = 0
concat_df.loc[concat_df['subject_id'].isin( moderate_pairs.preterm_ID.values), 'group'] = 1
concat_df.loc[concat_df['subject_id'].isin( extreme_pairs.matched_ID_with_outcome.values), 'group'] = 2
concat_df.loc[concat_df['subject_id'].isin( moderate_pairs.matched_ID_with_outcome.values), 'group'] = 3


### mean the L/R asymmetries 
pairs = [['M1L-Brainstem', 'M1R-Brainstem'],
        ['M1L-CaudL', 'M1R-CaudR'], 
        ['M1L-LentiL', 'M1R-LentiR'],
        ['M1L-ParacentralL', 'M1R-ParacentralR'],
        ['M1L-ThalfusL', 'M1R-ThalfusR'], 
         ['ParacentralL-Brainstem', 'ParacentralR-Brainstem'],
         ['ParacentralL-CaudL', 'ParacentralR-CaudR'],
         ['ParacentralL-LentiL', 'ParacentralR-LentiR'],
         ['ParacentralL-ThalfusL', 'ParacentralR-ThalfusR'],
         ['S1L-Brainstem', 'S1R-Brainstem'],
         ['S1L-CaudL', 'S1R-CaudR'], 
        ['S1L-LentiL', 'S1R-LentiR'],
        ['S1L-ParacentralL', 'S1R-ParacentralR'],
        ['S1L-ThalfusL', 'S1R-ThalfusR'], 
         ['S1L-M1L', 'S1R-M1R']]

for pair in pairs:
    for metric in ['FA', 'MD', 'RD', 'AD', 'NDI_post', 'NDI_pre', 'ODI_post', 'ODI_pre']:
        
        new_values = (concat_df[pair[0]+'_'+metric].values + concat_df[pair[1]+'_'+metric].values)/2
        
        label = pair[0].replace('L-', '-')
        if label[-1] == 'L':
            label = label[:-1]
      
        concat_df[label+'_'+metric] = new_values
        
        #concat_df.drop(columns=[pair[0]+'_'+metric, pair[1]+'_'+metric], inplace=True)
        
        
for pair in pairs:
    for metric in ['FA', 'MD', 'RD', 'AD', 'NDI_post', 'NDI_pre', 'ODI_post', 'ODI_pre']:
   
        concat_df.drop(columns=pair[0]+'_'+metric, inplace=True)
        concat_df.drop(columns=pair[1]+'_'+metric, inplace=True)
        
        
# add WM mean values /metric 
#### I NEED TO EXTRACT WHOLE BRAIN NODDI THINGS,but for now, keep it the same

ex_WM = pd.read_csv('../../DerivedData/extreme_pairs_mean_diffusion_metrics_over_WM.csv', index_col=0)
mod_WM = pd.read_csv('../../DerivedData/moderate_pairs_mean_diffusion_metrics_over_WM.csv', index_col=0)

for i, row in ex_WM.iterrows():
    #### I NEED TO EXTRACT WHOLE BRAIN NODDI THINGS,but for now, keep it the same
    for metric in ['FA', 'MD', 'RD', 'AD', 'NDI', 'ODI']: 
        
        if  metric in ['NDI', 'ODI']:
            concat_df.loc[concat_df.subject_id == row.preterm_ID, 'meanWM_'+metric+'_pre'] = row['preterm_'+metric]
            concat_df.loc[concat_df.subject_id == row.preterm_ID, 'meanWM_'+metric+'_post'] = row['preterm_'+metric+'_post']
            
            concat_df.loc[concat_df.subject_id == row.matched_ID, 'meanWM_'+metric+'_pre'] = row['control_'+metric]
            concat_df.loc[concat_df.subject_id == row.matched_ID, 'meanWM_'+metric+'_post'] = row['control_'+metric+'_post']
            
        else: 
            concat_df.loc[concat_df.subject_id == row.preterm_ID, 'meanWM_'+metric] =row['preterm_'+metric]
            concat_df.loc[concat_df.subject_id == row.matched_ID, 'meanWM_'+metric] =row['control_'+metric]
            

for i, row in mod_WM.iterrows():
    #### I NEED TO EXTRACT WHOLE BRAIN NODDI THINGS,but for now, keep it the same
    for metric in ['FA', 'MD', 'RD', 'AD', 'NDI', 'ODI']: 
        
        if  metric in ['NDI', 'ODI']:
            concat_df.loc[concat_df.subject_id == row.preterm_ID, 'meanWM_'+metric+'_pre'] = row['preterm_'+metric]
            concat_df.loc[concat_df.subject_id == row.preterm_ID, 'meanWM_'+metric+'_post'] = row['preterm_'+metric+'_post']
            
            concat_df.loc[concat_df.subject_id == row.matched_ID, 'meanWM_'+metric+'_pre'] = row['control_'+metric]
            concat_df.loc[concat_df.subject_id == row.matched_ID, 'meanWM_'+metric+'_post'] = row['control_'+metric+'_post']
            
        else: 
            concat_df.loc[concat_df.subject_id == row.preterm_ID, 'meanWM_'+metric] =row['preterm_'+metric]
            concat_df.loc[concat_df.subject_id == row.matched_ID, 'meanWM_'+metric] =row['control_'+metric]

# add Clinical variables 
clinic = pd.read_csv('../../DerivedData/Global.csv', sep=';')

to_keep = ['ParticipantID', 'FetalGrowthRestriction', 'PretermMorbiditiesx4', 'ParenteralNutrition>21d', 'Pregnancy-size', 
          'Sex', 'Gabirth', 'PMA-MRI']

clinic = clinic[to_keep]
clinic.rename(columns={'ParticipantID':'subject_id'}, inplace=True)

concat_df = pd.merge(concat_df, clinic, how="outer", on=["subject_id"])

Number of evaluated bundles: 33
Number of subject pairs: 59


/volatile/miniconda3/envs/ohbm/lib/python3.7/site-packages/ipykernel_launcher.py:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


In [4]:
#for metric in  ['FA', 'MD', 'RD', 'AD', 'NDI_post', 'NDI_pre', 'ODI_pre', 'ODI_post']:
for metric in  ['MD', 'RD', 'AD', 'NDI_post', 'ODI_post', 'FA']:
#for metric in ['FA']:
    metric_cols = [col for col in concat_df.columns if metric in col and 'meanWM' not in col]  ### and subject id

    for el in ['subject_id', 'group','meanWM_'+metric, 'FetalGrowthRestriction', 'PretermMorbiditiesx4', 'ParenteralNutrition>21d', 'Pregnancy-size', 
          'Sex', 'Gabirth', 'PMA-MRI']:
        metric_cols.append(el)
    
    new_df = concat_df[metric_cols].copy()
    new_df = pd.melt(new_df, id_vars=['subject_id', 'group','meanWM_'+metric, 'FetalGrowthRestriction', 'PretermMorbiditiesx4', 'ParenteralNutrition>21d', 'Pregnancy-size', 
          'Sex', 'Gabirth', 'PMA-MRI'], 
                 value_vars=[col for col in concat_df.columns if metric in col and 'meanWM' not in col])
    
    new_df.rename(columns={'meanWM_'+metric : 'meanWM'}, inplace=True)
    
    for col in ['Gabirth', 'PMA-MRI']:
        new_df[col] = new_df[col].astype(str)
        new_df[col] = new_df[col].apply(lambda x: x.replace(',','.'))
        new_df[col] = new_df[col].astype(np.float16)
    

    #to_drop = ['S1-Paracentral_ODI_post_' + metric, 'S1-M1_' + metric, 'M1-Paracentral_' + metric, 
    #       'ParacentralL-ParacentralR_'+ metric]
    
    # drop cortico-cortical that involve Paracentral 
    to_drop = ['S1-Paracentral_ODI_post_' + metric,  'M1-Paracentral_' + metric, 
           'ParacentralL-ParacentralR_'+ metric]
    
    if metric in ['NDI_post', 'ODI_post']:
        metric = metric.split('_')[0]
    
    new_df = new_df[~new_df.variable.isin(to_drop)]
    new_df.rename(columns={'variable': 'tract_cat'}, inplace=True)
    new_df.to_csv('../../DerivedData/for_jessica/{}_for_mancova_mergedLR_and_clinical.csv'.format(metric))

In [48]:
new_df.rename(columns={'PretermMorbiditiesx4':  'PretermMorbidities',
                      'ParenteralNutrition>21d': 'ParenteralNutrition',
                      'Pregnancy-size': 'PregnancySize', 
                      'PMA-MRI': 'PMAatMRI'}, inplace=True)

In [24]:
#mod = smf.ols(formula='Lottery ~ Literacy + Wealth + Region', data=df)
mod = smf.ols(formula='value ~ variable + group + meanWM + PMAatMRI + Sex + Gabirth+ FetalGrowthRestriction + PretermMorbidities +  ParenteralNutrition + PregnancySize  + variable*group + variable*Sex + group*Sex + variable*Sex*group', data=new_df, sub= new_df['subject_id'], wfactors=new_df['variable'])

res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.896
Model:                            OLS   Adj. R-squared:                  0.888
Method:                 Least Squares   F-statistic:                     114.4
Date:                Thu, 27 Jan 2022   Prob (F-statistic):               0.00
Time:                        16:47:20   Log-Likelihood:                 2793.5
No. Observations:                1116   AIC:                            -5429.
Df Residuals:                    1037   BIC:                            -5033.
Df Model:                          78                                         
Covariance Type:            nonrobust                                         
                                                                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------

In [25]:
#Sum of squares
print(anova_lm(res, typ=1, robust='hc3'))

                            df    sum_sq   mean_sq           F        PR(>F)
variable                  17.0  3.509057  0.206415  489.202160  0.000000e+00
Sex                        1.0  0.008926  0.008926   21.155616  4.757210e-06
FetalGrowthRestriction     1.0  0.002206  0.002206    5.227136  2.243805e-02
PretermMorbidities         1.0  0.021568  0.021568   51.114932  1.641815e-12
ParenteralNutrition        1.0  0.002944  0.002944    6.976851  8.381288e-03
variable:Sex              17.0  0.001137  0.000067    0.158493  9.999667e-01
group                      1.0  0.040511  0.040511   96.010707  9.692794e-22
variable:group            17.0  0.025148  0.001479    3.505950  1.948972e-06
group:Sex                  1.0  0.011204  0.011204   26.552911  3.068608e-07
variable:Sex:group        17.0  0.005032  0.000296    0.701457  8.036209e-01
meanWM                     1.0  0.123263  0.123263  292.132779  6.800716e-58
PMAatMRI                   1.0  0.009485  0.009485   22.479950  2.419841e-06